# Week 5 Homework: The Responsible Backtest

> *"A backtest is not a prediction. It is a prosecution brief — written by the same person who committed the crime."*
> *-- Marcos Lopez de Prado, paraphrased*

Your mission: build the infrastructure that separates a real trading signal from a lucky draw. By the end of this homework, you will have constructed three things — a leakage-free cross-validator, a transaction cost accounting layer, and a three-layer evaluation report — that together answer the only question a portfolio manager cares about: *should I allocate capital to this strategy?*

This is not a theoretical exercise. The pipeline you build here is, structurally, the same pipeline that a first-year quant researcher at Point72 or Two Sigma would produce before presenting a new signal to a PM. The specific tools differ (they use proprietary data, institutional TC models, and multi-year track records), but the logic is identical: purge, cost, evaluate, decide.

Here is what you will deliver:

## Deliverables

1. **Deliverable 1 -- PurgedKFold Cross-Validator**: A production-quality class that implements label-aware purging and embargo, verified correct by a zero-leakage invariant, with a visual diagnostic and an IC comparison against sklearn's TimeSeriesSplit.

2. **Deliverable 2 -- TransactionCostModel Pipeline**: A three-component cost accounting layer (spread + market impact + slippage) that converts gross portfolio returns into net returns across three spread regimes (optimistic, base, pessimistic), with a correctness test, equity curves, and a cost decomposition report.

3. **Deliverable 3 -- The Responsible Backtest Report**: A three-layer evaluation that integrates Deliverables 1 and 2 with CPCV, BHY multiple-testing correction, the deflated Sharpe ratio, and minimum track record length to produce a deployment verdict for the Week 4 alpha model.

In [ ]:
import sys
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests
import quantstats as qs

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

sys.path.insert(0, "code")
from data_setup import (
    CACHE_DIR, PLOT_DIR, SEED,
    load_alpha_output, load_ls_portfolio,
    load_ohlcv_data, load_mcap_tiers,
)

# Shared utilities
sys.path.insert(0, str(Path("code").resolve().parents[1]))
from course.shared.metrics import deflated_sharpe_ratio, ic_summary
from course.shared.backtesting import (
    sharpe_ratio, max_drawdown, cumulative_returns,
)
from course.shared.temporal import CombinatorialPurgedCV

np.random.seed(SEED)

---

# Deliverable 1: Build a Purged CV Engine

Every backtest you will run for the rest of this course — and for the rest of your career — depends on having a correct temporal splitting mechanism. Standard k-fold cross-validation shuffles observations randomly. Time-series-aware splits like sklearn's `TimeSeriesSplit` respect order but not labels. When your label is a 21-day forward return, the training sample at time $T-1$ already "knows" most of what the test sample at time $T$ will discover, because the label windows overlap. Purging removes these contaminated observations; embargoing blocks the information-rich boundary region after each test fold.

You are going to build this infrastructure from first principles. The algorithm is short — the class is under 80 lines. The stakes are high: a single leaking training sample can inflate your cross-validated IC enough to change which model you deploy. The correctness invariant is non-negotiable: zero overlapping training labels in the test period.

In production at a systematic fund, this class is part of the research library — the piece of code that researchers cannot override or bypass. If the purge logic has a bug, every signal evaluation in the firm is contaminated. Think of it as the plumbing: invisible when it works, catastrophic when it fails.

**Your deliverables for D1:**
1. Implement a `PurgedKFold` class with `split(X, y)` that yields `(train_idx, test_idx)` pairs with label-aware purging and embargo
2. Pass the zero-leakage correctness test on synthetic data (k=5 and k=10)
3. Visualize the train/test/purged/embargo zones on a timeline plot
4. Compare IC estimates between `TimeSeriesSplit` (with gap) and your `PurgedKFold` on the Week 4 alpha predictions

### Student Workspace

In [ ]:
# TODO: Define PurgedKFold class with __init__(n_splits, label_duration, embargo)
# Hint: label_duration is in trading days, embargo is in trading days
# The class should validate inputs (ValueError for negative or too-small values)
# Follow sklearn BaseCrossValidator interface: split(X, y=None, groups=None)

In [ ]:
# TODO: Implement the split logic
# Core algorithm:
# 1. Divide data into k sequential test folds
# 2. For each fold, remove training samples whose label window overlaps the test period
# 3. Additionally remove samples in the embargo zone after the test fold
# 4. Yield (train_indices, test_indices) for each fold

The split logic is the heart of the purging mechanism. For each fold, you need to identify the boundary where training labels overlap the test window and remove those contaminated samples. The embargo zone adds an extra buffer after the test period to guard against autocorrelation in the target variable -- even if a training observation's label does not formally overlap, the serial dependence of financial returns means nearby observations carry correlated information.

In [ ]:
# TODO: Run correctness test on synthetic data
# Create a synthetic DataFrame with 60 months of data
# Verify: zero leaking samples, zero embargo violations for k=5 and k=10

The correctness test is the non-negotiable gate before any downstream analysis. Construct synthetic data with known properties and verify that the purging invariant holds: zero training observations with labels that extend into the test period. If this invariant fails on synthetic data, every IC estimate and every Sharpe ratio downstream is contaminated. Run the test at both k=5 and k=10 to verify that the purge logic generalizes across fold counts.

In [ ]:
# TODO: Visualize the split timeline and compare IC across CV methods
# Generate a horizontal bar chart showing train/test/purged/embargo zones
# Compute fold-level IC for both TimeSeriesSplit and PurgedKFold

---

## ━━━ SOLUTION: Deliverable 1 ━━━

### PurgedKFold: Class Definition

The class starts simple — three validated parameters and the sklearn interface. The `n_splits`, `label_duration`, and `embargo` parameters define the splitting behavior entirely. Any negative duration or an `n_splits` below 2 raises a `ValueError` immediately, before any data touches the splitter. This fail-fast pattern is standard in production ML libraries — silent misconfiguration is the most dangerous kind.

In [ ]:
class PurgedKFold:
    """K-Fold CV that purges label-overlapping training samples.

    Ref: De Prado (2018), AFML Ch. 7.
    Follows sklearn BaseCrossValidator interface.
    """

    def __init__(self, n_splits: int = 5, label_duration: int = 21,
                 embargo: int = 5) -> None:
        if n_splits < 2:
            raise ValueError(f"n_splits must be >= 2, got {n_splits}")
        if label_duration < 0:
            raise ValueError(f"label_duration must be >= 0, got {label_duration}")
        if embargo < 0:
            raise ValueError(f"embargo must be >= 0, got {embargo}")

        self.n_splits = n_splits
        self.label_duration = label_duration
        self.embargo = embargo

    def get_n_splits(self, X=None, y=None, groups=None) -> int:
        """Return the number of splitting iterations."""
        return self.n_splits

Notice the design decision: `label_duration` and `embargo` are specified in calendar days, not data periods. This matters when the data frequency does not match the label frequency — monthly data with 21-day labels, for instance, needs the class to estimate how many periods correspond to 21 days. A production implementation would accept an explicit frequency parameter; ours infers it from the DatetimeIndex.

### PurgedKFold: Split Dispatcher

The `split` method dispatches to one of two implementations depending on whether the input has a `DatetimeIndex`. DateTime splitting gives precise, date-based purging. Positional splitting is the fallback for numpy arrays or integer-indexed DataFrames — it treats `label_duration` and `embargo` as row counts.

In [ ]:
def split(self, X, y=None, groups=None):
    """Yield (train_idx, test_idx) with purging and embargo."""
    if hasattr(X, "index") and isinstance(X.index, pd.DatetimeIndex):
        yield from self._split_datetime(X)
    else:
        yield from self._split_positional(X)

PurgedKFold.split = split

This dispatcher pattern — detect the index type and route to the appropriate implementation — is a recurring design in financial ML libraries. The datetime path is more complex but more accurate; the positional path is simpler but assumes uniform spacing.

### PurgedKFold: Frequency Estimation

Before we can purge, we need to know how many data periods correspond to a given number of calendar days. On daily data, one day is one period. On weekly data, one day is roughly 0.2 periods. On monthly data, one day is about 0.033 periods. This helper estimates the ratio from the index itself.

In [ ]:
def _estimate_periods_per_day(self, dates: pd.DatetimeIndex) -> float:
    """Estimate sampling frequency: average periods per calendar day."""
    if len(dates) < 2:
        return 1.0
    total_days = (dates[-1] - dates[0]).days
    if total_days <= 0:
        return 1.0
    return (len(dates) - 1) / total_days

PurgedKFold._estimate_periods_per_day = _estimate_periods_per_day

### PurgedKFold: DateTime Setup

The setup method computes fold parameters from the DatetimeIndex. The key computation is `label_end_dates` — for each observation at date $t$, this is $t + \text{label\_duration}$ days. Any training observation whose `label_end_date` falls at or after the test fold's start date has leaked future information into its label.

In [ ]:
def _datetime_setup(self, X):
    """Compute fold params for datetime splitting, or return None."""
    dates = X.index
    n = len(dates)
    if n < self.n_splits:
        raise ValueError(
            f"Cannot split {n} samples into {self.n_splits} folds."
        )

    periods_per_day = self._estimate_periods_per_day(dates)
    label_periods = max(1, round(self.label_duration * periods_per_day))

    seed = label_periods + 1
    if n <= seed:
        return None

    usable = n - seed
    fold_size = usable // self.n_splits
    label_end_dates = dates + pd.Timedelta(days=self.label_duration)
    embargo_td = pd.Timedelta(days=self.embargo)

    return dates, n, seed, fold_size, label_end_dates, embargo_td

PurgedKFold._datetime_setup = _datetime_setup

The `seed` variable reserves the first `label_periods + 1` observations as an initial training buffer. Without this, the first test fold would have zero training data after purging — a common bug in naive implementations.

### PurgedKFold: Single Fold Computation

This is the core of the algorithm. For a given fold $k$: identify the test indices, compute the purge boundary (where training labels stop overlapping the test period), compute the embargo boundary (the buffer after the test period), and build the training set from everything outside these zones.

In [ ]:
def _datetime_fold_indices(dates, n, k, seed, fold_size, n_splits,
                           label_end_dates, embargo_td):
    """Compute (train_idx, test_idx) for one datetime fold, or None."""
    test_start_idx = seed + k * fold_size
    test_end_idx = test_start_idx + fold_size if k < n_splits - 1 else n
    if test_start_idx >= n:
        return None

    test_idx = np.arange(test_start_idx, min(test_end_idx, n))
    if len(test_idx) == 0:
        return None

    test_start_date = dates[test_start_idx]
    test_end_date = dates[min(test_end_idx - 1, n - 1)]
    embargo_end_date = test_end_date + embargo_td

    purge_start_idx = np.searchsorted(label_end_dates, test_start_date, side="left")
    embargo_end_idx = min(np.searchsorted(dates, embargo_end_date, side="right"), n)

    before_purge = np.arange(0, purge_start_idx)
    after_embargo = np.arange(embargo_end_idx, n)
    train_idx = np.concatenate([before_purge, after_embargo])

    if len(train_idx) == 0 or len(test_idx) == 0:
        return None
    return train_idx, test_idx

Read that `purge_start_idx` line carefully: it uses `np.searchsorted` on `label_end_dates` to find the first observation whose label window reaches into the test period. Everything from that index onward (up to the test fold) is purged. This is the single most important line in the entire class — get it wrong and every IC estimate downstream is contaminated.

### PurgedKFold: DateTime Split Generator

The generator iterates over folds and yields the (train, test) pairs computed by the fold helper. On monthly data with 1-month forward-return labels, the purge zone is typically a single observation per fold boundary — structurally small, but the principle is universal.

In [ ]:
def _split_datetime(self, X):
    """Split using DatetimeIndex for precise date-based purging."""
    setup = self._datetime_setup(X)
    if setup is None:
        return
    dates, n, seed, fold_size, label_end_dates, embargo_td = setup

    for k in range(self.n_splits):
        result = _datetime_fold_indices(
            dates, n, k, seed, fold_size, self.n_splits,
            label_end_dates, embargo_td,
        )
        if result is not None:
            yield result

PurgedKFold._split_datetime = _split_datetime

### PurgedKFold: Positional Fallback

The positional path treats `label_duration` and `embargo` as row counts directly. Same logic, simpler computation — no date arithmetic needed. Useful when the input is a numpy array or when the DatetimeIndex is synthetic.

In [ ]:
def _positional_fold_indices(n, k, seed, fold_size, n_splits,
                             label_duration, embargo):
    """Compute (train_idx, test_idx) for one positional fold, or None."""
    test_start = seed + k * fold_size
    test_end = test_start + fold_size if k < n_splits - 1 else n
    if test_start >= n:
        return None

    test_idx = np.arange(test_start, min(test_end, n))
    if len(test_idx) == 0:
        return None

    purge_start = max(0, test_start - label_duration)
    embargo_end = min(n, test_end + embargo)

    before_purge = np.arange(0, purge_start)
    after_embargo = np.arange(embargo_end, n)
    train_idx = np.concatenate([before_purge, after_embargo])

    if len(train_idx) == 0:
        return None
    return train_idx, test_idx

In [ ]:
def _split_positional(self, X):
    """Positional fallback: label_duration and embargo as row counts."""
    n = len(X)
    if n < self.n_splits:
        raise ValueError(
            f"Cannot split {n} samples into {self.n_splits} folds."
        )

    seed = self.label_duration + 1
    if n <= seed:
        return

    usable = n - seed
    fold_size = usable // self.n_splits

    for k in range(self.n_splits):
        result = _positional_fold_indices(
            n, k, seed, fold_size, self.n_splits,
            self.label_duration, self.embargo,
        )
        if result is not None:
            yield result

PurgedKFold._split_positional = _split_positional

### Correctness Test: The Non-Negotiable Invariant

This is the test that must pass before anything else matters. We construct a synthetic DataFrame with 60 monthly periods, set a 21-day label window and 5-day embargo, and verify that every training sample in every fold has a label that ends before the test period starts. Zero leaking samples. Zero embargo violations. No exceptions.

In [ ]:
def _check_one_fold(train_idx, test_idx, dates, label_days, embargo_days):
    """Return (n_leaking, n_embargo_violations) for a single fold."""
    test_start_date = dates[test_idx[0]]
    test_end_date = dates[test_idx[-1]]
    embargo_end_date = test_end_date + pd.Timedelta(days=embargo_days)

    train_dates = dates[train_idx]
    label_end_dates = train_dates + pd.Timedelta(days=label_days)

    pre_test = train_dates < test_start_date
    leaking = (pre_test & (label_end_dates >= test_start_date)).sum()
    in_embargo = (train_dates > test_end_date) & (train_dates <= embargo_end_date)

    return leaking, in_embargo.sum()

The single-fold checker examines one train/test partition at a time: it reconstructs the label end dates for all training observations and counts how many of them reach into the test period. The separation into a per-fold helper and an aggregator follows the standard test design pattern -- test the unit (one fold), then test the system (all folds). If the per-fold invariant holds everywhere, the global invariant holds by construction.

In [ ]:
def _check_folds_for_leakage(folds, dates, label_days, embargo_days):
    """Aggregate leak/embargo counts across all folds."""
    n_leaking = 0
    n_embargo_violations = 0
    n_train_total = 0

    for train_idx, test_idx in folds:
        leak, emb = _check_one_fold(
            train_idx, test_idx, dates, label_days, embargo_days,
        )
        n_leaking += leak
        n_embargo_violations += emb
        n_train_total += len(train_idx)

    return n_leaking, n_embargo_violations, n_train_total

In [ ]:
def run_correctness_test(n_months: int = 60, label_days: int = 21,
                         embargo_days: int = 5, k: int = 5) -> dict:
    """Verify purging and embargo correctness on synthetic data."""
    dates = pd.date_range("2015-01-31", periods=n_months, freq="ME")
    X_synth = pd.DataFrame(
        {"feature": np.random.default_rng(SEED).standard_normal(n_months)},
        index=dates,
    )

    pkf = PurgedKFold(n_splits=k, label_duration=label_days, embargo=embargo_days)
    folds = list(pkf.split(X_synth))

    n_leaking, n_embargo_violations, n_train_total = _check_folds_for_leakage(
        folds, dates, label_days, embargo_days,
    )

    return {
        "n_folds": len(folds),
        "n_train_total": n_train_total,
        "n_leaking": n_leaking,
        "n_embargo_violations": n_embargo_violations,
        "passed": n_leaking == 0 and n_embargo_violations == 0,
    }

corr_k5 = run_correctness_test(n_months=60, label_days=21, embargo_days=5, k=5)
corr_k10 = run_correctness_test(n_months=60, label_days=21, embargo_days=5, k=10)

print(f"k=5:  n_folds={corr_k5['n_folds']}, n_leaking={corr_k5['n_leaking']}, "
      f"n_embargo_violations={corr_k5['n_embargo_violations']}, "
      f"PASSED={corr_k5['passed']}")
print(f"k=10: n_folds={corr_k10['n_folds']}, n_leaking={corr_k10['n_leaking']}, "
      f"n_embargo_violations={corr_k10['n_embargo_violations']}, "
      f"PASSED={corr_k10['passed']}")

Both tests pass: zero leaking training samples, zero embargo violations, exact fold counts. The `PurgedKFold` class is provably correct on this synthetic data. The invariant we verified — no training observation has a label window that extends into the test period — is the mathematical definition of purging from Chapter 7 of Lopez de Prado (2018). If this invariant holds, the cross-validated IC cannot be inflated by label leakage.

In production, this correctness test runs as part of the CI/CD pipeline every time the library is modified. A broken purge would silently invalidate every signal evaluation at the firm. The test is cheap (60 observations, runs in milliseconds) and the cost of failure is enormous.

### Visual Diagnostic: Seeing the Purge Zones

A timeline visualization makes the purging mechanism concrete. Each horizontal bar represents one fold. Blue regions are training data, orange is the test fold, red marks the purged zone (training observations whose labels overlap the test period), and gray is the embargo buffer.

We use weekly data (260 periods) for this visualization because monthly data (68 periods) would make the purge zone — which spans only one observation at monthly frequency — invisible on the plot. This adaptation is deliberate: the algorithm works the same at any frequency, but the visual must be legible.

In [ ]:
def plot_split_timeline(n_periods: int = 260, label_days: int = 21,
                        embargo_days: int = 5, k: int = 5,
                        freq: str = "W-MON",
                        save_path=None) -> plt.Figure:
    """Visualize train / test / purged / embargo zones for k folds."""
    dates = pd.date_range("2015-01-05", periods=n_periods, freq=freq)
    X_dt = pd.DataFrame({"dummy": 0}, index=dates)
    pkf = PurgedKFold(n_splits=k, label_duration=label_days,
                      embargo=embargo_days)
    splits = list(pkf.split(X_dt))

    fig, ax = plt.subplots(figsize=(14, 4))

    colors = {
        "train": "#4C72B0",
        "test": "#DD8452",
        "purged": "#C44E52",
        "embargo": "#8C8C8C",
        "unused": "#EEEEEE",
    }

    n = len(dates)
    all_idx = np.arange(n)

    total_days = max(1, (dates[-1] - dates[0]).days)
    periods_per_day = (n - 1) / total_days
    embargo_periods = max(1, round(embargo_days * periods_per_day))

    for fold_num, (train_idx, test_idx) in enumerate(splits):
        y = fold_num

        test_set = set(test_idx)
        train_set = set(train_idx)

        test_start_idx = test_idx[0]
        test_end_idx = test_idx[-1]
        test_start_date = dates[test_start_idx]

        label_end_dates = dates + pd.Timedelta(days=label_days)

        for idx in all_idx:
            if idx in test_set:
                color = colors["test"]
            elif idx < test_start_idx and label_end_dates[idx] >= test_start_date:
                color = colors["purged"]
            elif idx > test_end_idx and idx <= test_end_idx + embargo_periods:
                color = colors["embargo"]
            elif idx in train_set:
                color = colors["train"]
            else:
                color = colors["unused"]

            ax.barh(y, width=1, left=idx, color=color, height=0.7,
                    linewidth=0, edgecolor="none")

    ax.set_yticks(range(len(splits)))
    ax.set_yticklabels([f"Fold {i + 1}" for i in range(len(splits))])
    ax.set_xlabel("Week Index")
    ax.set_title(
        f"PurgedKFold Split Visualization (k={k}, "
        f"label={label_days}d, embargo={embargo_days}d, weekly data)"
    )

    legend_patches = [
        mpatches.Patch(color=colors["train"], label="Train"),
        mpatches.Patch(color=colors["test"], label="Test"),
        mpatches.Patch(color=colors["purged"], label="Purged"),
        mpatches.Patch(color=colors["embargo"], label="Embargo"),
        mpatches.Patch(color=colors["unused"], label="Unused"),
    ]
    ax.legend(handles=legend_patches, loc="upper right", fontsize=8)
    plt.tight_layout()
    plt.show()

    return fig

fig_splits = plot_split_timeline(
    n_periods=260, label_days=21, embargo_days=5, k=5, freq="W-MON",
)

Look at the red (purged) zones — they appear just before each test fold (orange), spanning approximately 3 weeks of data. Those are the training observations whose 21-day forward-return labels would have "peeked" into the test period. Without purging, those observations train the model on information it should not have access to. The gray embargo zones after each test fold provide an additional buffer against autocorrelation in the target variable.

On daily data with 21-day labels, the purge zone would be 21 observations wide — a substantial fraction of the data near each fold boundary. On monthly data with 1-month labels, the purge zone shrinks to roughly a single observation. This frequency dependence is not a bug — it is the structural reality of how label overlap scales with data granularity.

### IC Comparison: TimeSeriesSplit vs. PurgedKFold

The practical question is: does purging change the IC estimate? We run both TimeSeriesSplit (with `gap=1` to skip one observation, sklearn's built-in leakage mitigation) and PurgedKFold (with full label-aware purging and 5-day embargo) on the Week 4 GBM alpha predictions, computing mean cross-sectional Spearman IC per fold.

In [ ]:
def _compute_ic_for_date(predictions, d):
    """Compute cross-sectional Spearman IC for one date, or None."""
    if d not in predictions.index.get_level_values("date"):
        return None
    pred_d = predictions.loc[d]["prediction"].dropna()
    actual_d = predictions.loc[d]["actual"].dropna()
    common = pred_d.index.intersection(actual_d.index)
    if len(common) < 10:
        return None
    ic_val = float(pred_d[common].corr(actual_d[common], method="spearman"))
    return ic_val if np.isfinite(ic_val) else None

The per-date IC helper isolates the cross-sectional correlation computation into a reusable unit. This matters because IC evaluation requires at least 10 common tickers between predictions and actuals on any given date -- dates with too few stocks produce unreliable rank correlations. The helper returns `None` for such dates, and the aggregation layer downstream handles missing values gracefully. This defensive pattern prevents a single illiquid month from corrupting the fold-level IC estimate.

In [ ]:
def compute_fold_ics(predictions: pd.DataFrame, splits, dates: pd.Index
                     ) -> list:
    """Mean cross-sectional Spearman IC per fold's test period."""
    fold_ics = []
    for train_idx, test_idx in splits:
        test_dates = dates[test_idx]
        ic_vals = []
        for d in test_dates:
            ic_val = _compute_ic_for_date(predictions, d)
            if ic_val is not None:
                ic_vals.append(ic_val)
        fold_ics.append(np.mean(ic_vals) if ic_vals else np.nan)
    return fold_ics

In [ ]:
def compare_cv_methods(predictions: pd.DataFrame, k: int = 10) -> dict:
    """Compare TimeSeriesSplit vs. PurgedKFold on IC computation."""
    dates = predictions.index.get_level_values("date").unique().sort_values()
    n = len(dates)

    tss = TimeSeriesSplit(n_splits=k, gap=1)
    X_pos = np.arange(n).reshape(-1, 1)
    wf_splits = list(tss.split(X_pos))
    wf_ics = compute_fold_ics(predictions, wf_splits, dates)

    X_dt = pd.DataFrame({"dummy": 0}, index=dates)
    pkf = PurgedKFold(n_splits=k, label_duration=21, embargo=5)
    purged_splits = list(pkf.split(X_dt))
    purged_ics = compute_fold_ics(predictions, purged_splits, dates)

    wf_clean = [v for v in wf_ics if np.isfinite(v)]
    purged_clean = [v for v in purged_ics if np.isfinite(v)]

    wf_mean = float(np.mean(wf_clean)) if wf_clean else np.nan
    purged_mean = float(np.mean(purged_clean)) if purged_clean else np.nan
    delta = wf_mean - purged_mean

    t_stat = np.nan
    if len(wf_clean) >= 2 and len(purged_clean) >= 2:
        t_stat, _ = stats.ttest_ind(wf_clean, purged_clean)

    return {
        "wf_ics": wf_ics, "purged_ics": purged_ics,
        "wf_mean": wf_mean, "purged_mean": purged_mean,
        "delta": delta,
        "t_stat": float(t_stat) if np.isfinite(t_stat) else np.nan,
        "n_wf": len(wf_clean), "n_purged": len(purged_clean),
    }

alpha = load_alpha_output()
predictions = alpha["predictions"]
results = compare_cv_methods(predictions, k=10)

print(f"Walk-Forward IC:   {results['wf_mean']:.4f}  (n_folds={results['n_wf']})")
print(f"PurgedKFold IC:    {results['purged_mean']:.4f}  (n_folds={results['n_purged']})")
print(f"Delta (WF - PKF):  {results['delta']:+.4f}")
print(f"t-stat:            {results['t_stat']:.3f}")

The IC delta is -0.0011 — essentially zero. Walk-forward IC is 0.0203, PurgedKFold IC is 0.0213. The direction is slightly reversed from expectations (purged IC marginally higher), and the t-statistic of -0.030 confirms this difference is statistically meaningless. This is the correct result for monthly data with 1-month forward-return labels: the label window approximately equals the data frequency, so purging removes at most one observation per fold boundary. The contamination zone is structurally compressed.

This is itself the insight. On daily data with 21-day forward returns — the standard setup at production quant shops — Lopez de Prado (2018, Chapters 7-8) documents that walk-forward overestimates IC by 20-50% relative to purged CV. On our monthly data, the effect vanishes. Purging's importance scales with the ratio of label duration to data period: the higher that ratio, the larger the leakage zone.

In [ ]:
def plot_ic_comparison(results: dict) -> plt.Figure:
    """Bar chart comparing mean IC (WF vs. PurgedKFold) with std error bars."""
    wf_ics = [v for v in results["wf_ics"] if np.isfinite(v)]
    purged_ics = [v for v in results["purged_ics"] if np.isfinite(v)]

    means = [np.mean(wf_ics), np.mean(purged_ics)]
    stds = [np.std(wf_ics), np.std(purged_ics)]
    labels = ["TimeSeriesSplit\n(with leakage)", "PurgedKFold\n(cleaned)"]
    colors = ["#4C72B0", "#DD8452"]

    fig, ax = plt.subplots(figsize=(6, 5))
    bars = ax.bar(labels, means, yerr=stds, capsize=6, color=colors,
                  alpha=0.85, edgecolor="black", linewidth=0.7)
    ax.axhline(0, color="black", linewidth=0.8, linestyle="--")
    ax.set_ylabel("Mean Fold IC (Spearman)")
    ax.set_title("CV Method Comparison: TimeSeriesSplit vs. PurgedKFold\n"
                 "(error bars = std across folds)")

    for bar, mean, std in zip(bars, means, stds):
        ax.text(bar.get_x() + bar.get_width() / 2, mean + std + 0.002,
                f"{mean:.4f}", ha="center", va="bottom", fontsize=10,
                fontweight="bold")

    delta = results["delta"]
    ax.text(0.5, 0.02, f"\u0394 IC (WF \u2212 Purged) = {delta:+.4f}",
            ha="center", va="bottom", transform=ax.transAxes,
            fontsize=9, color="darkred",
            bbox=dict(boxstyle="round,pad=0.3", fc="lightyellow", alpha=0.8))

    plt.tight_layout()
    plt.show()

    return fig

fig_ic = plot_ic_comparison(results)

The bar chart confirms what the numbers told us: the two bars are indistinguishable in height, and the error bars (standard deviation across folds) dwarf the delta. The IC difference is noise at this data frequency. But the class is correct — the zero-leakage invariant proves that — and on daily data with multi-day labels, the gap would be substantial.

Building the purging mechanism from scratch confirms you understand the algorithm. Correctness is verified by the zero-leakage invariant, not by the IC gap magnitude. The IC gap depends on data frequency and label duration; the algorithm's correctness does not.

Now you have the CV infrastructure. The next question: even if the signal survives purged evaluation, can it survive the cost of trading?

---

# Deliverable 2: Transaction Cost Accounting Pipeline

A backtest without transaction costs is a wish. Every time your portfolio trades — every rebalance, every new position, every exit — the market charges you. The bid-ask spread is the explicit tax: you buy at the ask, sell at the bid, and the difference goes to the market maker. Market impact is the implicit tax: your trade moves the price against you, and the bigger the trade relative to daily volume, the larger the impact. At the turnover levels of a monthly cross-sectional strategy, these costs compound into an annual drag that can erase half the gross alpha.

This deliverable builds the accounting layer that converts gross portfolio returns into net returns, decomposed by cost component. The `TransactionCostModel` you build here generalizes Week 4's simple `net_returns(gross, turnover, cost_bps)` — which applied a flat per-unit cost — into a three-component decomposition with stock-level granularity. Think of it as the difference between estimating shipping costs as "10% of revenue" versus computing actual freight charges per item per destination.

The three spread regimes you will test — optimistic (3 bps, reflecting the best-case for liquid large-caps), base (market-cap tiered at 5/15 bps), and pessimistic (25 bps, covering illiquid or stress-period execution) — are how production funds bracket their cost assumptions. No single number is correct; the regime analysis tells the PM where the strategy breaks.

**Your deliverables for D2:**
1. Implement a `TransactionCostModel` class with `fit(ohlcv)` and `transform(gross_returns)` methods
2. Pass the 2-asset correctness test (analytical turnover and spread cost match)
3. Run the model across three spread regimes and report gross vs. net Sharpe
4. Visualize equity curves, cost decomposition, and the top-5 highest-cost months

### Student Workspace

In [ ]:
# TODO: Define TransactionCostModel class
# Attributes: weights, spread_bps, impact_coeff
# Methods: fit(ohlcv) precomputes daily vol and ADV; transform(gross) computes costs
# The class should decompose TC into spread and market impact components

In [ ]:
# TODO: Implement the correctness check
# 2-asset portfolio shifting from [1, 0] to [0, 1]:
# turnover should be exactly 1.0 (100% one-way)
# spread_cost should be 2 * spread_bps / 10_000 (round-trip on full weight change)

Before running the model on hundreds of stocks and real portfolio weights, verify it on a trivially solvable case. The 2-asset analytical test has a known answer: a portfolio that shifts from `[1, 0]` to `[0, 1]` has exactly 100% one-way turnover and a spread cost of exactly $2 \times \text{spread\_bps} / 10{,}000$. If the model cannot reproduce these numbers to machine precision, there is a bug in the weight differencing, the basis-point scaling, or the turnover computation.

In [ ]:
# TODO: Run three-regime analysis (optimistic / base / pessimistic)
# Plot equity curves and cost decomposition
# Report gross vs. net Sharpe for each regime

---

## ━━━ SOLUTION: Deliverable 2 ━━━

### TransactionCostModel: Class Definition

The class stores portfolio weights, a spread assumption (either a scalar for flat-rate cost or a per-stock Series for tiered costs), and the Almgren-Chriss market impact coefficient $\eta$. After `fit()` precomputes rolling volatility and average daily volume from OHLCV data, `transform()` walks through each rebalance period computing turnover, spread cost, and impact cost.

In [ ]:
class TransactionCostModel:
    """Transaction cost model: spread + market impact (Almgren-Chriss).

    Spread = sum(|dw|) * half_spread_bps / 10_000.
    Impact = eta * sigma * sqrt(participation) per stock.
    """

    def __init__(
        self,
        weights: pd.DataFrame,
        spread_bps: float | pd.Series,
        impact_coeff: float = 0.1,
    ) -> None:
        self.weights = weights.copy()
        self.spread_bps = spread_bps
        self.impact_coeff = impact_coeff
        self.daily_vol: pd.DataFrame | None = None
        self.adv: pd.DataFrame | None = None
        self.turnover: pd.Series | None = None
        self.spread_cost: pd.Series | None = None
        self.impact_cost: pd.Series | None = None
        self.net_returns: pd.Series | None = None

The design separates `fit()` (precompute market data features) from `transform()` (apply cost model to returns). This mirrors the sklearn estimator pattern and allows the same fitted model to be applied to different return series — useful for sensitivity analysis.

### TransactionCostModel: fit() — Market Data Preprocessing

The `fit()` method computes two rolling statistics from daily OHLCV data: 30-day rolling daily volatility (the $\sigma$ in the impact formula) and 30-day average daily dollar volume (the denominator of the participation rate). Both use `min_periods=10` to avoid NaN gaps at the start.

In [ ]:
ASSUMED_AUM = 100_000_000  # $100M — standard research portfolio assumption
DEFAULT_DAILY_VOL = 0.01   # ~1% daily vol fallback for missing tickers

def fit(self, ohlcv: pd.DataFrame) -> "TransactionCostModel":
    """Precompute 30-day rolling daily vol and ADV from OHLCV."""
    close = ohlcv["Close"]
    volume = ohlcv["Volume"]

    daily_ret = close.pct_change()
    self.daily_vol = daily_ret.rolling(30, min_periods=10).std()

    dollar_vol = close * volume
    self.adv = dollar_vol.rolling(30, min_periods=10).mean()

    return self

TransactionCostModel.fit = fit

At a real fund, these statistics come from a market data vendor (Bloomberg, Refinitiv) at much higher granularity — intraday VWAP, realized volatility from tick data, actual fill rates from the execution desk. Our proxy approach (rolling 30-day statistics from daily OHLCV) is standard for research with free data.

### TransactionCostModel: Spread Cost Computation

The spread cost for a single rebalance period is the sum of absolute weight changes multiplied by the half-spread, in basis points. When the spread is a scalar, this is a simple multiplication. When it is a per-stock Series (for tiered costs), we reindex to match the active stocks and apply stock-level costs.

In [ ]:
def _compute_spread_cost(abs_delta, half_spread):
    """Compute spread cost from absolute weight changes."""
    if np.isscalar(half_spread):
        return abs_delta.sum() * half_spread / 10_000
    spread_per_stock = (
        half_spread.reindex(abs_delta.index).fillna(half_spread.mean())
    )
    return (abs_delta * spread_per_stock).sum() / 10_000

### TransactionCostModel: Market Impact via Square-Root Law

The Almgren-Chriss (2000) square-root market impact model says: the temporary price impact of a trade scales as $\eta \cdot \sigma \cdot \sqrt{Q/V}$, where $\sigma$ is daily volatility, $Q$ is trade size in dollars, $V$ is average daily volume in dollars, and $\eta$ is an empirical coefficient (typically 0.1-0.2).

The intuition: if you trade 1% of daily volume, impact is manageable. If you trade 25% of daily volume, you are moving the market against yourself. The square root means doubling your trade size does not double the impact — it increases it by about 41%. This concavity is why large funds split orders across multiple days.

In [ ]:
def _compute_impact_cost(abs_delta, date, daily_vol, adv, impact_coeff):
    """Compute Almgren-Chriss market impact for one period."""
    if daily_vol is None or adv is None:
        return 0.0

    active = abs_delta[abs_delta > 1e-6].index
    if len(active) == 0:
        return 0.0

    vol_snap = (
        daily_vol.loc[:date].tail(1).squeeze()
        if date in daily_vol.index
        else daily_vol.iloc[-1]
    )
    vol_snap = vol_snap.reindex(active).fillna(DEFAULT_DAILY_VOL)

    adv_snap = (
        adv.loc[:date].tail(1).squeeze()
        if date in adv.index
        else adv.iloc[-1]
    )
    adv_snap = adv_snap.reindex(active).fillna(adv_snap.median())
    adv_snap = adv_snap.replace(0, np.nan).fillna(adv_snap.median())

    trade_dollars = abs_delta[active] * ASSUMED_AUM
    participation = (trade_dollars / adv_snap).clip(upper=1.0)
    impact_per_stock = impact_coeff * vol_snap * np.sqrt(participation)
    return float((impact_per_stock * abs_delta[active]).sum())

Our model uses $\eta = 0.1$, the lower end of the 0.1-0.2 production range. At $100M AUM and 5% ADV participation rate, this produces 3-8 bps impact per trade for large-cap S&P 500 stocks — consistent with Almgren and Chriss's original calibration. At institutional scale ($1B+), participation rates increase and impact costs can exceed spread costs.

### TransactionCostModel: Per-Period Step and Transform

Each period computes: the weight delta from the previous period, one-way turnover (half the sum of absolute deltas), spread cost, and impact cost. Net return is gross minus total costs.

In [ ]:
def _compute_period_step(date, weights, half_spread, daily_vol, adv,
                         impact_coeff, gross_ret):
    """Compute turnover, costs, and net return for a single period."""
    prev_w = weights.loc[weights.index[weights.index.get_loc(date) - 1]]
    curr_w = weights.loc[date]
    delta_w = (curr_w - prev_w).reindex(weights.columns).fillna(0.0)
    abs_delta = delta_w.abs()
    turnover = abs_delta.sum() / 2.0

    spread_cost = _compute_spread_cost(abs_delta, half_spread)
    impact_cost = _compute_impact_cost(
        abs_delta, date, daily_vol, adv, impact_coeff,
    )
    net_ret = gross_ret - spread_cost - impact_cost
    return turnover, spread_cost, impact_cost, net_ret

In [ ]:
def _to_series(lst, col):
    """Convert list of dicts to a DatetimeIndex Series."""
    df = pd.DataFrame(lst).set_index("date")
    df.index = pd.DatetimeIndex(df.index)
    return df[col]

def transform(self, gross_returns: pd.Series) -> "TransactionCostModel":
    """Compute per-period TC components and net returns."""
    if self.daily_vol is None:
        import warnings
        warnings.warn(
            "TransactionCostModel.transform() called without fit(). "
            "Market impact will be zero (no vol/ADV data)."
        )
    weights = self.weights
    trade_dates = weights.index[1:]
    common_dates = trade_dates.intersection(gross_returns.index)

    turnover_vals, spread_vals, impact_vals, net_vals = [], [], [], []

    for date in common_dates:
        to, sc, ic, nr = _compute_period_step(
            date, weights, self.spread_bps,
            self.daily_vol, self.adv, self.impact_coeff,
            gross_returns.loc[date],
        )
        turnover_vals.append({"date": date, "turnover": to})
        spread_vals.append({"date": date, "spread_cost": sc})
        impact_vals.append({"date": date, "impact_cost": ic})
        net_vals.append({"date": date, "net_return": nr})

    self.turnover = _to_series(turnover_vals, "turnover")
    self.spread_cost = _to_series(spread_vals, "spread_cost")
    self.impact_cost = _to_series(impact_vals, "impact_cost")
    self.net_returns = _to_series(net_vals, "net_return")
    return self

TransactionCostModel.transform = transform

### TransactionCostModel: Report Method

The `report()` method prints a structured TC summary with all key metrics, flags high turnover, and identifies the five most expensive months with their dominant cost component.

In [ ]:
def report(self, label: str = "regime") -> None:
    """Print structured TC summary; flag high-turnover periods."""
    mean_to = self.turnover.mean()
    ann_net_sharpe = sharpe_ratio(self.net_returns, periods_per_year=12)
    ann_net_ret = self.net_returns.mean() * 12
    mdd = max_drawdown(self.net_returns)

    print(f"  TC Report [{label}]")
    print(f"    Mean one-way monthly turnover : {mean_to:.1%}")
    if mean_to > 0.50:
        print(f"    HIGH TURNOVER: {mean_to:.1%} — TC drag is material")
    print(f"    Annualized net return         : {ann_net_ret:.2%}")
    print(f"    Annualized net Sharpe         : {ann_net_sharpe:.3f}")
    print(f"    Max drawdown (net)             : {mdd:.2%}")
    print(f"    Mean monthly spread cost      : {self.spread_cost.mean():.6f}")
    print(f"    Mean monthly impact cost      : {self.impact_cost.mean():.6f}")

    total_tc = self.spread_cost + self.impact_cost
    top5 = total_tc.nlargest(5)
    print(f"    Top-5 highest-cost months:")
    for dt, cost in top5.items():
        dom = "spread" if self.spread_cost[dt] >= self.impact_cost[dt] else "impact"
        print(f"      {dt.date()}: total={cost:.6f}  dominant={dom}")

TransactionCostModel.report = report

### Tiered Spread Builder

Market-cap tiers map directly to spread assumptions: large-cap stocks (top tercile by market cap) get 5 bps half-spread, mid-cap and small-cap get 15 bps. These numbers reflect current post-decimalization spreads for S&P 500 constituents — large-cap spreads are 2-5 bps in current markets (frec.com 2023, Anderson UCLA), while mid-cap spreads run 12-25 bps (Corwin & Schultz, 2012).

In [ ]:
def build_tiered_spread(mcap_tiers: pd.Series, tickers: list) -> pd.Series:
    """Map market-cap tier to half-spread in bps."""
    tier_map = {"large": 5.0, "mid": 15.0, "small": 15.0}
    spread = mcap_tiers.reindex(tickers).map(tier_map).fillna(15.0)
    return spread

### Correctness Check: The 2-Asset Analytical Test

Before running on real data, we verify the cost model against an analytically solvable case: a 2-asset portfolio that shifts from [1, 0] to [0, 1] in one period. The one-way turnover must be exactly 1.0 (100%), and the spread cost must equal exactly $2 \times \text{spread\_bps} / 10{,}000$, because both assets trade their full weight.

In [ ]:
def correctness_check(spread_bps: float = 10.0) -> dict:
    """2-asset [1,0]->[0,1] analytical check for TransactionCostModel."""
    dates = pd.date_range("2020-01-31", periods=2, freq="ME")
    w = pd.DataFrame(
        {"ASSET_A": [1.0, 0.0], "ASSET_B": [0.0, 1.0]},
        index=dates,
    )
    gross = pd.Series([0.01], index=dates[1:])

    tcm = TransactionCostModel(weights=w, spread_bps=spread_bps)
    tcm.transform(gross)

    expected_spread = 2.0 * spread_bps / 10_000
    actual_turnover = float(tcm.turnover.iloc[0])
    actual_spread = float(tcm.spread_cost.iloc[0])
    match = abs(actual_spread - expected_spread) < 1e-10

    return {
        "turnover": actual_turnover,
        "spread_cost": actual_spread,
        "expected_spread_cost": expected_spread,
        "match": match,
    }

chk = correctness_check(spread_bps=10.0)
print(f"Correctness check: {'PASS' if chk['match'] else 'FAIL'}")
print(f"  turnover = {chk['turnover']:.4f} (expected 1.0)")
print(f"  spread_cost = {chk['spread_cost']:.8f} "
      f"(expected {chk['expected_spread_cost']:.8f})")

The correctness check passes: turnover is exactly 1.0000, and spread cost matches the analytical formula to machine precision. This confirms the cost accounting has no sign errors, no off-by-one bugs in the weight differencing, and no scaling issues in the basis-point conversion.

### Three-Regime TC Analysis on Real Data

Now we apply the model to the real Week 4 long-short portfolio across three spread regimes. This is the analysis a quant researcher produces when answering the PM's question: "what happens to this strategy's profitability under different cost assumptions?"

In [ ]:
def run_three_regimes(
    weights: pd.DataFrame,
    gross_returns: pd.Series,
    ohlcv: pd.DataFrame,
    mcap_tiers: pd.Series,
) -> dict:
    """Run TransactionCostModel across three spread regimes."""
    tickers = weights.columns.tolist()
    tiered_spread = build_tiered_spread(mcap_tiers, tickers)

    regimes = {
        "optimistic": 3.0,
        "base": tiered_spread,
        "pessimistic": 25.0,
    }

    results = {}
    for name, spread in regimes.items():
        print(f"  Running TC model [{name}]...")
        tcm = TransactionCostModel(weights=weights, spread_bps=spread)
        tcm.fit(ohlcv)
        tcm.transform(gross_returns)
        results[name] = tcm

    return results

ohlcv = load_ohlcv_data()
mcap_tiers = load_mcap_tiers()
ls_pf = load_ls_portfolio()
weights = ls_pf["weights"]
gross_returns = ls_pf["gross_returns"]

regimes = run_three_regimes(weights, gross_returns, ohlcv, mcap_tiers)

for name, tcm in regimes.items():
    tcm.report(label=name)

The numbers tell a story. Mean monthly one-way turnover is approximately 140% — nearly every position is replaced each month as the alpha scores re-rank 449 stocks. Under the optimistic regime (3 bps flat), net Sharpe is 0.778 — only a modest reduction from gross 0.871. Under the base regime (tiered 5/15 bps), net Sharpe drops to 0.676 — still above the typical institutional hurdle of 0.50. Under the pessimistic regime (25 bps flat), net Sharpe falls to 0.409 — below the hurdle. The strategy is viable for large-cap execution but breaks at mid-cap cost levels.

This is the three-regime sensitivity table in the format a PM would actually see on a strategy evaluation memo. The optimistic column says "best case if everything goes right and fills are perfect." The pessimistic column says "worst case during stressed markets or illiquid names." The base column is the operating assumption. A PM who sees the pessimistic Sharpe below hurdle knows the strategy has limited capacity and no room for execution error.

### Equity Curve Visualization

The equity curve comparison makes the progressive drag visible. The gross line (black) shows the portfolio's raw alpha. The three net lines show what is left after costs erode the returns at each assumption level.

In [ ]:
def plot_equity_curves(
    gross_returns: pd.Series,
    regimes: dict,
) -> tuple:
    """Gross vs. net equity curves across three spread regimes."""
    fig, ax = plt.subplots(figsize=(11, 5))

    gross_cum = (1 + gross_returns).cumprod()
    ax.plot(gross_cum.index, gross_cum.values, "k-", linewidth=2, label="Gross")

    colors = {"optimistic": "#2ca02c", "base": "#1f77b4", "pessimistic": "#d62728"}
    for name, tcm in regimes.items():
        net_cum = (1 + tcm.net_returns).cumprod()
        ax.plot(net_cum.index, net_cum.values, color=colors[name],
                linewidth=1.5, label=f"Net ({name})")

    ax.set(
        title="Gross vs. Net Equity Curves — Three Spread Regimes",
        xlabel="Date",
        ylabel="Cumulative Return (starting at 1.0)",
    )
    ax.legend(loc="upper left")
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    return fig, ax

fig_eq, ax_eq = plot_equity_curves(gross_returns, regimes)

Notice how the gap between gross and net widens over time — this is compounding at work. A 2% annual drag does not just subtract 2% from the final value; it compounds against you for the entire holding period. Over 5 years, a strategy with 10% gross return and 2% TC drag does not earn 40% net; it earns closer to 34%. That 6% difference is the compound effect of costs — the same exponential arithmetic that makes compound returns so powerful in the other direction.

The top-5 highest-cost months reveal which market conditions are most expensive: March 2020 (COVID crash) shows impact-dominated costs because volatility spiked and the Almgren-Chriss impact term is proportional to $\sigma$. In normal periods, spread costs dominate. This decomposition tells the execution desk where to focus optimization: during stress, slow down order routing; during calm markets, tighten the spread target.

### TC Decomposition Over Time

The stacked area chart shows the monthly cost breakdown between spread and impact. The visual pattern is clear: impact costs (orange) spike during high-volatility regimes while spread costs (blue) remain more stable.

In [ ]:
def plot_tc_decomposition(regime, label: str) -> tuple:
    """Stacked area chart: spread cost + impact cost per month."""
    fig, ax = plt.subplots(figsize=(11, 4))

    dates = regime.spread_cost.index
    spread = regime.spread_cost.values
    impact = regime.impact_cost.values

    ax.stackplot(
        dates,
        spread,
        impact,
        labels=["Spread cost", "Market impact"],
        colors=["#1f77b4", "#ff7f0e"],
        alpha=0.8,
    )
    ax.set(
        title=f"TC Decomposition Over Time — {label}",
        xlabel="Date",
        ylabel="Monthly TC drag (decimal)",
    )
    ax.legend(loc="upper right")
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    return fig, ax

fig_dc, ax_dc = plot_tc_decomposition(regimes["base"], label="Base Regime")

The March 2020 spike is unmistakable — total monthly TC drag exceeds 0.6% in that single month, driven almost entirely by impact. The Almgren-Chriss model captures this correctly: when daily volatility doubles during a crisis, impact costs for the same trade size also double. A production execution desk would react by slowing execution, spreading orders across multiple days, and using VWAP algorithms to minimize impact. Our research model assumes instantaneous execution — another source of optimism in the backtest.

### Top-5 Highest-Cost Months

The horizontal bar chart identifies the five most expensive months and decomposes each into spread versus impact.

In [ ]:
def plot_top5_cost_months(regime, label: str) -> tuple:
    """Horizontal bar chart of top-5 highest-cost months."""
    total_tc = regime.spread_cost + regime.impact_cost
    top5 = total_tc.nlargest(5).sort_values()

    spread_top5 = regime.spread_cost.reindex(top5.index)
    impact_top5 = regime.impact_cost.reindex(top5.index)

    month_labels = [d.strftime("%Y-%m") for d in top5.index]

    fig, ax = plt.subplots(figsize=(8, 4))
    y_pos = range(len(top5))
    ax.barh(y_pos, spread_top5.values, color="#1f77b4", label="Spread")
    ax.barh(y_pos, impact_top5.values, left=spread_top5.values,
            color="#ff7f0e", label="Impact")
    ax.set_yticks(list(y_pos))
    ax.set_yticklabels(month_labels)
    ax.set(
        title=f"Top-5 Highest-Cost Months — {label}",
        xlabel="Monthly TC drag (decimal)",
    )
    ax.legend()
    plt.tight_layout()
    plt.show()
    return fig, ax

fig_t5, ax_t5 = plot_top5_cost_months(regimes["base"], label="Base Regime")

All five highest-cost months cluster around two events: the COVID crash (March-April 2020) and the 2022 rate shock (June, August, October 2022). These are exactly the periods where a PM would want to know the cost story. In production, the execution desk tracks these months in real time and feeds the measured shortfall back to the research team to recalibrate the TC model — what we are building here is the pre-trade version of what execution quants measure post-trade.

You now have the cost accounting layer. The next deliverable integrates it with the purged CV from D1, the deflated Sharpe ratio, and CPCV to produce the complete deployment verdict.

---

# Deliverable 3: The Responsible Backtest Report

A real fund researcher does not just report a Sharpe ratio and a cumulative return chart. They produce a structured evaluation that a PM and risk committee can interrogate. The report has three layers, each one adding a harder test for the strategy to pass. Layer 1 is the performance report: how does the strategy perform net of costs? Layer 2 is the overfitting assessment: is the strategy genuinely the best model, or did it win a beauty contest in noise? Layer 3 is the deployment readiness check: how long would you need to run the strategy live before you could confirm the observed Sharpe is real?

The goal is not to make the strategy look good. The goal is to make the truth clear. If the strategy cannot survive three layers of honest evaluation, the backtest just saved the fund a year of live losses.

**Your deliverables for D3:**
1. **Layer 1:** Tear sheet with Sharpe, Sortino, max drawdown, Calmar, CAGR. DSR at M=10 with DEPLOY/NO-DEPLOY verdict.
2. **Layer 2:** CPCV across 3 model variants (GBM, NN, Ridge). PBO with PROCEED/OVERFIT verdict. BHY multiple-testing correction. Winning model selection with full justification.
3. **Layer 3:** Minimum track record length at 95% confidence. Capital allocation discussion with paper-trading recommendation.

### Student Workspace

In [ ]:
# TODO: Layer 1 — Compute performance metrics for gross and net returns
# Include: Sharpe, Sortino, Calmar, CAGR, max drawdown, skewness, excess kurtosis
# Generate a quantstats tearsheet (HTML or inline plots)
# Compute DSR at M=10 and issue DEPLOY/NO-DEPLOY verdict

In [ ]:
# TODO: Layer 2 — Run CPCV across model variants
# Build IC series for GBM, NN, and Ridge
# Compute PBO from CPCV paths
# Apply BHY correction to IC t-statistics
# Select winning model with full justification

Layer 2 asks whether the model you selected is genuinely superior, or whether it won an in-sample competition against noise. CPCV generates every possible train/test partition and checks whether the in-sample winner retains its rank out-of-sample. The BHY correction controls the false discovery rate when multiple models are tested on the same data -- without it, the probability of declaring at least one model significant by chance grows linearly with the number of models tried.

In [ ]:
# TODO: Layer 3 — Compute MinTRL
# Apply the Bailey-Lopez de Prado MinTRL formula
# Assess whether observed track record is sufficient
# Write capital allocation recommendation

---

## ━━━ SOLUTION: Deliverable 3 ━━━

### Load Upstream Results

Deliverable 3 integrates the outputs of D1 and D2 with additional statistical evaluation tools. We load the net returns from D2's base regime (tiered 5/15 bps) and the alpha predictions from Week 4 for the CPCV analysis.

In [ ]:
_d2_results = pd.read_parquet(CACHE_DIR / "d2_tc_model_results.parquet")

net_returns_base = _d2_results["net_return_base"]
gross_returns_d3 = _d2_results["gross_return"]
turnover_series = _d2_results["turnover"]

In [ ]:
_alpha = load_alpha_output()
gbm_predictions = _alpha["predictions"]
forward_returns = _alpha["forward_returns"]
gbm_ic_series = _alpha["ic_series"]["ic"]

nn_predictions = _alpha.get("nn_predictions", None)
nn_ic_series = _alpha.get("nn_ic_series", None)
if nn_ic_series is not None and isinstance(nn_ic_series, pd.DataFrame):
    nn_ic_series = nn_ic_series["ic"]

expanded_features = _alpha.get("expanded_features", None)
if expanded_features is None:
    feat_file = CACHE_DIR / "expanded_features_w5.parquet"
    if feat_file.exists():
        expanded_features = pd.read_parquet(feat_file)

print("Data loaded:")
print(f"  Net returns (base): {len(net_returns_base)} months, "
      f"{net_returns_base.index[0].date()} to {net_returns_base.index[-1].date()}")
print(f"  GBM IC series: {len(gbm_ic_series)} months, "
      f"mean IC = {gbm_ic_series.mean():.4f}")
print(f"  NN predictions available: {nn_predictions is not None}")
print(f"  Expanded features available: {expanded_features is not None}")

All upstream caches are loaded. The net returns span roughly 67 months of out-of-sample performance, and the GBM model's mean IC is approximately 0.026 — weak by institutional standards but within the expected range for a monthly cross-sectional signal on a survivorship-biased S&P 500 universe. The question is whether this signal, after honest evaluation, is distinguishable from noise.

## Layer 1: Tear Sheet and DSR Verdict

### Performance Metrics

Layer 1 starts with the standard performance metrics that every fund report includes. We compute these for both the gross (naive, no TC) and net (base regime, purged CV) return series to make the degradation explicit.

In [ ]:
def compute_full_metrics(returns: pd.Series, label: str) -> dict:
    """Compute annualized performance metrics for a return series.

    Args:
        returns: Monthly return series.
        label: Name for display.

    Returns:
        dict with Sharpe, Sortino, max_dd, CAGR, skewness, excess_kurtosis.
    """
    ann_factor = 12
    mean_r = returns.mean()
    std_r = returns.std()

    sharpe = (mean_r * ann_factor) / (std_r * np.sqrt(ann_factor)) if std_r > 0 else 0.0

    downside = returns[returns < 0].std()
    sortino = (mean_r * ann_factor) / (downside * np.sqrt(ann_factor)) if downside > 0 else 0.0

    mdd = max_drawdown(returns)
    cagr = (1 + mean_r) ** ann_factor - 1
    calmar = abs(cagr / mdd) if mdd != 0 else 0.0

    skew = float(stats.skew(returns.dropna()))
    ekurt = float(stats.kurtosis(returns.dropna(), fisher=True))

    return {
        "label": label,
        "sharpe": sharpe,
        "sortino": sortino,
        "max_dd": mdd,
        "cagr": cagr,
        "calmar": calmar,
        "skewness": skew,
        "excess_kurtosis": ekurt,
        "n_months": len(returns.dropna()),
    }

gross_metrics = compute_full_metrics(gross_returns_d3, "Gross (naive)")
net_metrics = compute_full_metrics(net_returns_base, "Net base (purged)")

for m in [gross_metrics, net_metrics]:
    print(f"  {m['label']}:")
    print(f"    Sharpe={m['sharpe']:.3f}, Sortino={m['sortino']:.3f}, "
          f"Calmar={m['calmar']:.3f}")
    print(f"    CAGR={m['cagr']:.2%}, MaxDD={m['max_dd']:.2%}")
    print(f"    Skewness={m['skewness']:.3f}, ExcessKurt={m['excess_kurtosis']:.3f}")

The gross Sharpe of 0.871 drops to a net Sharpe of 0.676 — a 22% reduction from transaction costs alone. The max drawdown deepens from roughly -32% (gross) to -34% (net), because costs amplify the loss in every down month. The excess kurtosis of 4.23 tells us the return distribution has much heavier tails than a Gaussian — this will matter enormously when we compute the deflated Sharpe ratio below.

Notice the skewness: -0.26. Slightly left-skewed, meaning the distribution has a longer left tail. Combined with high kurtosis, this is the signature of equity long-short strategies: occasional large losses that Gaussian risk models cannot anticipate.

### Quantstats Tear Sheet

The quantstats library generates a publication-ready tear sheet with all standard performance analytics. We generate both the HTML tearsheet and inline matplotlib plots for the notebook.

In [ ]:
_qs_returns = net_returns_base.copy()
_qs_returns.name = "Net (base regime)"
_qs_returns.index = pd.DatetimeIndex(_qs_returns.index)

_qs_html_path = PLOT_DIR / "d3_quantstats_tearsheet.html"
qs.reports.html(_qs_returns, output=str(_qs_html_path),
                title="Layer 1 Tear Sheet — Net Returns (Base Regime)")
print(f"  quantstats HTML tearsheet saved -> {_qs_html_path}")

In [ ]:
fig_ts, axes_ts = plt.subplots(4, 1, figsize=(12, 14))
fig_ts.suptitle("Layer 1 Tear Sheet — Net Returns (Base Regime)", fontsize=13)

# 1. Cumulative returns
cum_net = (1 + net_returns_base).cumprod()
cum_gross = (1 + gross_returns_d3).cumprod()
ax = axes_ts[0]
ax.plot(cum_gross.index, cum_gross.values, label="Gross", color="steelblue", alpha=0.7)
ax.plot(cum_net.index, cum_net.values, label="Net (base)", color="darkorange", linewidth=2)
ax.set_title("Cumulative Returns")
ax.set_ylabel("Portfolio Value")
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Monthly return distribution
ax2 = axes_ts[1]
ax2.hist(net_returns_base.values, bins=20, color="darkorange", alpha=0.7,
         edgecolor="white")
ax2.axvline(0, color="black", linestyle="--", linewidth=1)
ax2.axvline(net_returns_base.mean(), color="red", linestyle="--",
            linewidth=1.5, label=f"Mean={net_returns_base.mean():.3f}")
ax2.set_title("Monthly Return Distribution (Net)")
ax2.set_xlabel("Return")
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Drawdown
cum_series = (1 + net_returns_base).cumprod()
rolling_max = cum_series.cummax()
drawdown_series = (cum_series - rolling_max) / rolling_max
ax3 = axes_ts[2]
ax3.fill_between(drawdown_series.index, drawdown_series.values, 0,
                 color="red", alpha=0.4, label="Drawdown")
ax3.set_title("Underwater Equity Curve (Net)")
ax3.set_ylabel("Drawdown")
ax3.set_xlabel("Date")
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Monthly returns heatmap
_monthly_ret = net_returns_base.copy()
_monthly_ret.index = pd.DatetimeIndex(_monthly_ret.index)
_pivot = _monthly_ret.groupby([_monthly_ret.index.year, _monthly_ret.index.month]).sum().unstack()
_pivot.columns = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                   "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][:_pivot.shape[1]]
ax4 = axes_ts[3]
_im = ax4.imshow(_pivot.values * 100, cmap="RdYlGn", aspect="auto",
                  vmin=-10, vmax=10)
ax4.set_xticks(range(_pivot.shape[1]))
ax4.set_xticklabels(_pivot.columns, fontsize=8)
ax4.set_yticks(range(_pivot.shape[0]))
ax4.set_yticklabels([str(int(y)) for y in _pivot.index], fontsize=8)
ax4.set_title("Monthly Returns Heatmap (%)")
for i in range(_pivot.shape[0]):
    for j in range(_pivot.shape[1]):
        if j < len(_pivot.columns) and not np.isnan(_pivot.values[i, j]):
            ax4.text(j, i, f"{_pivot.values[i, j]*100:.1f}",
                    ha="center", va="center", fontsize=7,
                    color="black" if abs(_pivot.values[i, j]) < 0.06 else "white")
plt.colorbar(_im, ax=ax4, shrink=0.8, label="Return (%)")

plt.tight_layout()
plt.show()

The four-panel tear sheet tells the full story at a glance. The cumulative returns panel (top) shows the gross-vs-net gap widening over time. The return distribution (second panel) shows the mild negative skew and fat tails that inflate the DSR penalty. The underwater curve (third panel) reveals a maximum drawdown of -34% — painful but not fatal for a long-short equity strategy. The monthly heatmap (bottom) shows returns scattered with no obvious seasonal pattern, which is what you want to see in a cross-sectional alpha strategy.

### Gross vs. Net Equity Curve (Layer 1 Capstone)

A dedicated equity curve comparison with IS/OOS annotation. The entire visible period is out-of-sample — the Week 4 model was trained on pre-2019 data that does not appear on this chart.

In [ ]:
fig_eq, ax_eq = plt.subplots(figsize=(11, 5))

ax_eq.plot(cum_gross.index, cum_gross.values, label="Gross (naive, no TC)",
           color="steelblue", alpha=0.7, linewidth=1.5)
ax_eq.plot(cum_net.index, cum_net.values, label="Net (base regime, purged)",
           color="darkorange", linewidth=2)

ax_eq.annotate(
    "IS: model training\n(pre-2019-04, not shown)",
    xy=(cum_gross.index[0], cum_gross.iloc[0]),
    xytext=(cum_gross.index[min(6, len(cum_gross) - 1)], cum_gross.max() * 0.55),
    fontsize=9, color="gray", fontstyle="italic",
    arrowprops=dict(arrowstyle="->", color="gray", lw=1.2),
    bbox=dict(boxstyle="round,pad=0.3", fc="lightyellow", alpha=0.8),
)
ax_eq.text(
    0.55, 0.92, "Full period is OOS (out-of-sample model deployment)",
    transform=ax_eq.transAxes, fontsize=9, color="green",
    bbox=dict(boxstyle="round,pad=0.3", fc="honeydew", alpha=0.8),
)

ax_eq.set_title("Layer 1: Gross vs. Net Equity Curve (OOS Period)")
ax_eq.set_xlabel("Date")
ax_eq.set_ylabel("Portfolio Value ($1 invested)")
ax_eq.legend(loc="upper left")
ax_eq.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

The chart clearly marks the entire visible period as OOS. The gross line reaches about $2.36 per dollar invested; the net line under base-regime costs reaches about $1.79. That $0.57 gap is the cost of doing business — the market's toll for moving money.

### DSR Verdict

The deflated Sharpe ratio adjusts the observed Sharpe for non-normality (via skewness and excess kurtosis) and multiple testing (via the number of strategy variants tried). At $M=10$ trials — a conservative estimate for a researcher who tested a handful of model configurations — the DSR tells us whether the observed Sharpe is statistically distinguishable from the best of 10 noise processes.

In [ ]:
_net = net_returns_base.dropna()
_skew = float(stats.skew(_net))
_ekurt = float(stats.kurtosis(_net, fisher=True))
_obs = len(_net)
_ann_sr = net_metrics["sharpe"]
_monthly_sr = _net.mean() / _net.std() if _net.std() > 0 else 0.0

N_TRIALS_L1 = 10
dsr_l1 = deflated_sharpe_ratio(
    observed_sr=_monthly_sr,
    n_trials=N_TRIALS_L1,
    n_obs=_obs,
    skew=_skew,
    excess_kurt=_ekurt,
)
deploy_verdict = "DEPLOY" if dsr_l1 >= 0.50 else "NO-DEPLOY"

print(f"Monthly SR: {_monthly_sr:.4f}")
print(f"DSR (M={N_TRIALS_L1}, T={_obs}): {dsr_l1:.4f}  -> {deploy_verdict}")
print(f"Skewness: {_skew:.4f}, ExcessKurt: {_ekurt:.4f}")

The DSR of 0.504 at $M=10$ is a razor-thin pass — barely above the 0.50 DEPLOY threshold. Under slightly different cost assumptions (as in the lecture's Section 6, where DSR is 0.414 using net-tiered returns with a different cost specification), the strategy would fail. This fragility is itself the teaching point: the DEPLOY/NO-DEPLOY boundary is not a sharp cliff but a zone of uncertainty that depends on how precisely you measure costs and how honestly you count the number of strategies you tried before landing on this one.

The excess kurtosis of 4.23 is the dominant force driving the DSR down. Under a Gaussian assumption (excess kurtosis = 0), the same Sharpe ratio over the same period would produce a DSR well above 0.90. The heavy tails in real equity returns inflate the denominator of the DSR formula through the $(\gamma_4+2)/4 \cdot \hat{SR}^2$ term, making it much harder to distinguish a genuine signal from noise. Non-normality is not a footnote in the DSR formula — it is the dominant term.

## Layer 2: CPCV, PBO, and BHY Correction

Layer 2 asks a deeper question: is the GBM model genuinely the best of the model variants, or did it win the in-sample beauty contest by fitting noise? We answer this using Combinatorial Purged Cross-Validation (CPCV) across three model variants (GBM, NN, Ridge) and measure the Probability of Backtest Overfitting (PBO).

### Ridge Walk-Forward Predictions

We need a third model variant for the CPCV analysis. The Week 4 cache provides GBM and NN; we synthesize Ridge predictions via walk-forward training with a 36-month rolling window and `StandardScaler` normalization.

In [ ]:
def _ridge_fold_predict(
    features: pd.DataFrame, targets: pd.Series,
    train_dates, test_date,
) -> list:
    """Train Ridge on one fold and return prediction records."""
    try:
        X_train = features.loc[train_dates]
        y_train = targets.reindex(X_train.index).dropna()
        X_train = X_train.loc[y_train.index]
    except KeyError:
        return []

    if len(X_train) < 50:
        return []

    X_test_idx = features.index[
        features.index.get_level_values("date") == test_date
    ]
    if len(X_test_idx) == 0:
        return []

    X_test = features.loc[X_test_idx]

    X_train_clean = X_train.dropna()
    y_train = y_train.reindex(X_train_clean.index).dropna()
    X_train_clean = X_train_clean.loc[y_train.index]

    if len(X_train_clean) < 50:
        return []

    X_test_filled = X_test.fillna(X_train_clean.median())

    scaler = StandardScaler()
    X_tr_sc = scaler.fit_transform(X_train_clean.values)
    X_te_sc = scaler.transform(X_test_filled.values)

    model = Ridge(alpha=1.0)
    model.fit(X_tr_sc, y_train.values)
    preds = model.predict(X_te_sc)

    return [
        {"date": idx[0], "ticker": idx[1], "prediction": preds[j]}
        for j, idx in enumerate(X_test_idx)
    ]

In [ ]:
def compute_ridge_predictions(
    features: pd.DataFrame, targets: pd.Series, train_window: int = 36
) -> pd.Series:
    """Walk-forward Ridge predictions with StandardScaler per fold."""
    dates = features.index.get_level_values("date").unique().sort_values()
    preds_list = []

    for i, test_date in enumerate(dates[train_window:], start=train_window):
        train_dates = dates[i - train_window: i]
        fold_preds = _ridge_fold_predict(features, targets, train_dates, test_date)
        preds_list.extend(fold_preds)

        if (i - train_window) % 12 == 0:
            print(f"  [Ridge] fold {i - train_window + 1}/{len(dates) - train_window}: "
                  f"{test_date.strftime('%Y-%m')}")

    if not preds_list:
        return pd.Series(dtype=float)

    df_p = pd.DataFrame(preds_list).set_index(["date", "ticker"])["prediction"]
    return df_p

if expanded_features is not None and len(expanded_features) > 0:
    oos_dates = gbm_predictions.index.get_level_values("date").unique()
    feat_dates = expanded_features.index.get_level_values("date").unique()
    common_dates = oos_dates.intersection(feat_dates)

    if len(common_dates) > 40:
        ridge_preds = compute_ridge_predictions(
            features=expanded_features,
            targets=forward_returns,
            train_window=36,
        )
        print(f"  Ridge predictions: {len(ridge_preds)} samples across "
              f"{ridge_preds.index.get_level_values('date').nunique()} dates")
    else:
        ridge_preds = pd.Series(dtype=float)
else:
    ridge_preds = pd.Series(dtype=float)

The Ridge model serves as a "noise baseline" — a linear model with L2 regularization that should capture any persistent linear relationships but miss the nonlinear patterns that GBM and NN exploit. If Ridge performs comparably to GBM, the nonlinear models are not adding value. If Ridge's IC is substantially weaker, the nonlinear signal is genuine.

### Per-Model IC Series

We compute monthly cross-sectional Spearman IC for each model variant — the metric that CPCV will operate on.

In [ ]:
def monthly_ic_from_predictions(pred_df: pd.DataFrame) -> pd.Series:
    """Compute monthly cross-sectional Spearman IC from a prediction DataFrame."""
    dates = pred_df.index.get_level_values("date").unique()
    ic_vals = {}
    for d in dates:
        slice_d = pred_df.xs(d, level="date")[["prediction", "actual"]].dropna()
        if len(slice_d) < 20:
            continue
        ic_vals[d] = float(
            slice_d["prediction"].corr(slice_d["actual"], method="spearman")
        )
    return pd.Series(ic_vals)

The first IC helper operates on a unified prediction DataFrame where `prediction` and `actual` columns are already aligned. The second variant below handles the case where predictions and actuals live in separate Series with a multi-level `(date, ticker)` index -- the format produced by the Ridge walk-forward loop, where predictions are generated independently of the GBM pipeline. Both helpers enforce the same minimum-20-ticker threshold per date to ensure the rank correlation is statistically meaningful.

In [ ]:
def monthly_ic_pred_vs_actuals(
    pred_series: pd.Series, actuals_multiindex: pd.Series
) -> pd.Series:
    """Compute monthly IC for predictions vs. a separate actuals series."""
    dates = pred_series.index.get_level_values("date").unique()
    ic_vals = {}
    for d in dates:
        try:
            pred_d = pred_series.xs(d, level="date").dropna()
        except KeyError:
            continue
        try:
            act_d = actuals_multiindex.xs(d, level="date").dropna()
        except KeyError:
            continue
        common = pred_d.index.intersection(act_d.index)
        if len(common) < 20:
            continue
        ic_vals[d] = float(pred_d[common].corr(act_d[common], method="spearman"))
    return pd.Series(ic_vals)

In [ ]:
gbm_ic_monthly = monthly_ic_from_predictions(gbm_predictions)
print(f"GBM IC: mean={gbm_ic_monthly.mean():.4f}, "
      f"t={gbm_ic_monthly.mean() / gbm_ic_monthly.std() * np.sqrt(len(gbm_ic_monthly)):.2f}")

if nn_predictions is not None:
    nn_ic_monthly = monthly_ic_from_predictions(nn_predictions)
    print(f"NN IC: mean={nn_ic_monthly.mean():.4f}, "
          f"t={nn_ic_monthly.mean() / nn_ic_monthly.std() * np.sqrt(len(nn_ic_monthly)):.2f}")
else:
    nn_ic_monthly = None
    print("NN predictions: unavailable")

if isinstance(ridge_preds, pd.Series) and len(ridge_preds) > 0:
    ridge_ic_monthly = monthly_ic_pred_vs_actuals(ridge_preds, forward_returns)
    if len(ridge_ic_monthly) > 5:
        print(f"Ridge IC: mean={ridge_ic_monthly.mean():.4f}, "
              f"t={ridge_ic_monthly.mean() / ridge_ic_monthly.std() * np.sqrt(len(ridge_ic_monthly)):.2f}")
    else:
        ridge_ic_monthly = None
else:
    ridge_ic_monthly = None

The GBM model shows mean IC of approximately 0.026 with a t-statistic around 1.39 — below the conventional 1.96 threshold for 5% significance, and far below the Harvey-Liu-Zhu hurdle of 3.0 that any new factor claim requires. The NN model shows similar IC around 0.024. The Ridge model, as expected, is substantially weaker with near-zero IC. None of the three models is individually statistically significant — this is the harsh reality of cross-sectional alpha on a 68-month sample with 174 tickers.

### CPCV and PBO Computation

Combinatorial Purged CV generates all possible train/test partitions from $k$ temporal groups. With $k=6$ and $n_\text{test}=2$, we get $\binom{6}{2} = 15$ train-test paths. For each path, we identify the in-sample best model and check whether it ranks above or below the median out-of-sample. PBO is the fraction of paths where the IS winner underperforms the OOS median.

In [ ]:
def _align_ic_matrix(
    model_ic_dict: dict, n_splits: int
) -> pd.DataFrame:
    """Align model IC series to common dates and build IC matrix."""
    ic_frames = {}
    for name, series in model_ic_dict.items():
        if series is not None and len(series) > 0:
            ic_frames[name] = series.sort_index()

    if len(ic_frames) < 2:
        return pd.DataFrame()

    common_idx = None
    for s in ic_frames.values():
        common_idx = s.index if common_idx is None else common_idx.intersection(s.index)

    ic_matrix = pd.DataFrame(
        {n: ic_frames[n].reindex(common_idx) for n in ic_frames}
    ).dropna()

    if len(ic_matrix) < n_splits + 2:
        return pd.DataFrame()

    return ic_matrix

In [ ]:
def compute_pbo_from_ic(
    model_ic_dict: dict, n_splits: int = 6, n_test_splits: int = 2
) -> tuple:
    """Compute Probability of Backtest Overfitting via CPCV."""
    ic_matrix = _align_ic_matrix(model_ic_dict, n_splits)
    if ic_matrix.empty:
        return float("nan"), []

    cpcv = CombinatorialPurgedCV(n_splits=n_splits, n_test_splits=n_test_splits,
                                  purge_gap=1)
    X_proxy = pd.DataFrame(index=ic_matrix.index)

    paths_below_median = 0
    pbo_records = []

    for train_idx, test_idx in cpcv.split(X_proxy):
        if len(train_idx) == 0 or len(test_idx) == 0:
            continue

        is_ic = ic_matrix.iloc[train_idx].mean()
        oos_ic = ic_matrix.iloc[test_idx].mean()

        is_winner = is_ic.idxmax()
        oos_rank_of_winner = int(
            oos_ic.rank(ascending=False)[is_winner]
        )
        oos_median_rank = (len(ic_matrix.columns) + 1) / 2.0

        below_median = oos_rank_of_winner > oos_median_rank
        if below_median:
            paths_below_median += 1

        pbo_records.append({
            "is_winner": is_winner,
            "oos_rank_winner": oos_rank_of_winner,
            "oos_ic_winner": float(oos_ic[is_winner]),
            "n_models": len(ic_matrix.columns),
            "oos_median_rank": oos_median_rank,
        })

    n_paths = len(pbo_records)
    pbo = paths_below_median / n_paths if n_paths > 0 else float("nan")
    return pbo, pbo_records

model_ic_dict = {"GBM": gbm_ic_monthly}
if nn_ic_monthly is not None:
    model_ic_dict["NN"] = nn_ic_monthly
if ridge_ic_monthly is not None:
    model_ic_dict["Ridge"] = ridge_ic_monthly

pbo, pbo_records = compute_pbo_from_ic(model_ic_dict)
print(f"PBO = {pbo:.4f} ({pbo * 100:.1f}% of paths where IS winner is OOS loser)")
print(f"N CPCV paths: {len(pbo_records)}")

PBO = 0.267 means the in-sample winner ranks above the OOS median in 73% of CPCV partitions. This falls in the "genuine alpha" sub-range of 0.25-0.35 documented by Bailey et al. (2015) — the in-sample best model tends to maintain its relative ranking out-of-sample. But with only 15 paths and 3 models, this estimate has high sampling variance. And crucially, PBO < 0.5 does not substitute for statistical significance: a model can consistently be the best of three weak models without any of them being distinguishable from zero.

### BHY Multiple-Testing Correction

The Benjamini-Hochberg-Yekutieli procedure controls the false discovery rate when testing multiple hypotheses on correlated data — exactly our situation with three model variants evaluated on the same returns.

In [ ]:
model_stats = {}
for name, ic_s in model_ic_dict.items():
    if ic_s is None or len(ic_s) < 10:
        continue
    n = len(ic_s)
    mean_ic = ic_s.mean()
    std_ic = ic_s.std()
    t_stat = mean_ic / (std_ic / np.sqrt(n)) if std_ic > 0 else 0.0
    p_val = 2 * (1 - stats.t.cdf(abs(t_stat), df=n - 1))
    model_stats[name] = {"mean_ic": mean_ic, "t_stat": t_stat, "p_value": p_val, "n": n}

In [ ]:
if len(model_stats) >= 2:
    stat_names = list(model_stats.keys())
    raw_pvals = np.array([model_stats[n]["p_value"] for n in stat_names])
    _, adj_pvals, _, _ = multipletests(raw_pvals, method="fdr_bh")
    for i, name in enumerate(stat_names):
        model_stats[name]["adj_p_value"] = adj_pvals[i]
        print(f"  {name}: t={model_stats[name]['t_stat']:.3f}, "
              f"p_raw={raw_pvals[i]:.4f}, p_BHY={adj_pvals[i]:.4f}")
else:
    for name in model_stats:
        model_stats[name]["adj_p_value"] = model_stats[name]["p_value"]

No model clears statistical significance after BHY correction. The GBM model's BHY-adjusted p-value of approximately 0.467 is far from the 0.05 threshold. This is the multiple testing reality: even before applying the Harvey-Liu-Zhu $t > 3.0$ hurdle, none of our models would pass a basic significance test at conventional levels. The signal is real enough to produce positive IC (the PBO confirms the ranking is stable), but not strong enough for statistical certainty on a 67-month sample.

### Winning Model Selection

We select the model with the highest mean IC among those passing BHY significance. Since no model clears the threshold, we fall back to best raw IC — the GBM model.

In [ ]:
significant = {n: s for n, s in model_stats.items() if s.get("adj_p_value", 1.0) < 0.10}
if significant:
    winning_model = max(significant, key=lambda n: significant[n]["mean_ic"])
    selection_basis = "BHY adj p < 0.10"
else:
    winning_model = max(model_stats, key=lambda n: model_stats[n]["mean_ic"])
    selection_basis = "best IC (no model cleared BHY threshold)"

winner_stats = model_stats[winning_model]
print(f"Winning model: {winning_model} ({selection_basis})")
print(f"IC: {winner_stats['mean_ic']:.4f}, t-stat: {winner_stats['t_stat']:.3f}")
print(f"PBO: {pbo:.4f}, DSR@M=10: {dsr_l1:.4f} -> {deploy_verdict}")

In [ ]:
comparison_rows = []
for name, s in model_stats.items():
    comparison_rows.append({
        "Model": name,
        "Mean IC": round(s["mean_ic"], 4),
        "t-stat": round(s["t_stat"], 3),
        "p_raw": round(s["p_value"], 4),
        "p_BHY": round(s.get("adj_p_value", s["p_value"]), 4),
        "Selected": "YES" if name == winning_model else "",
    })
comparison_df = pd.DataFrame(comparison_rows).set_index("Model")
print(comparison_df.to_string())

The model comparison table is the artifact a PM reviews. GBM leads with IC of 0.026 (t = 1.39), NN follows at 0.024, and Ridge trails near zero. No model clears BHY significance, but GBM is selected as the best available. The PBO of 0.267 provides modest comfort that the ranking is not an artifact.

### PBO Distribution Plot

The histogram of OOS ranks across CPCV paths visualizes the PBO result. Bars to the left of the median line indicate paths where the IS winner maintained its ranking; bars to the right indicate overfitting.

In [ ]:
if len(pbo_records) > 0:
    oos_ranks = [r["oos_rank_winner"] for r in pbo_records]
    n_models_cpcv = pbo_records[0]["n_models"]
    median_rank = (n_models_cpcv + 1) / 2.0

    fig_pbo, ax_pbo = plt.subplots(figsize=(8, 5))
    ax_pbo.hist(oos_ranks, bins=range(1, n_models_cpcv + 2), align="left",
                color="steelblue", alpha=0.7, edgecolor="white",
                label="OOS rank of IS winner")
    ax_pbo.axvline(median_rank, color="red", linestyle="--", linewidth=2,
                   label=f"Median rank = {median_rank:.1f}")
    ax_pbo.fill_betweenx([0, len(pbo_records) * 0.9], median_rank,
                          n_models_cpcv + 0.5, alpha=0.15, color="red",
                          label="Below-median region (overfitting)")
    ax_pbo.set_title(
        f"Layer 2: IS-Winner OOS Rank Distribution\nPBO = {pbo:.3f} ({len(pbo_records)} paths)"
    )
    ax_pbo.set_xlabel("OOS Rank of IS-Best Model (1=best)")
    ax_pbo.set_ylabel("Number of CPCV paths")
    ax_pbo.set_xticks(range(1, n_models_cpcv + 1))
    ax_pbo.legend()
    ax_pbo.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

The majority of the histogram mass sits at rank 1 (the IS winner remains the OOS winner) with a smaller fraction falling into the below-median (red) region. The PBO of 0.267 is visible as the proportion of bars in the shaded zone. With only 3 models and 15 paths, the distribution is coarse — at a production fund with 50 variants and $k=10$ (252 paths), the PBO estimate would be much more precise.

## Layer 3: MinTRL and Deployment Readiness

Layer 3 delivers the reality check. How long would you need to run this strategy live before confirming the observed Sharpe ratio with 95% confidence? The Bailey-Lopez de Prado (2014) Minimum Track Record Length formula accounts for the non-normality of returns — the same heavy tails that depressed our DSR.

### MinTRL Formula

The formula says: the minimum number of observations needed to confirm that the true Sharpe exceeds a benchmark (here, zero) at a given confidence level is:

$$\text{MinTRL} = \frac{z_\alpha^2 \cdot (1 - \gamma_3 \cdot \hat{SR}_m + \frac{\gamma_4}{4} \cdot \hat{SR}_m^2)}{(\hat{SR}_m - SR_0)^2}$$

where $\gamma_3$ is skewness, $\gamma_4$ is excess kurtosis, $\hat{SR}_m$ is the monthly Sharpe ratio, and $z_\alpha$ is the one-tailed normal quantile.

In [ ]:
def min_track_record_length(
    sharpe_annual: float,
    skewness: float,
    excess_kurtosis: float,
    confidence: float = 0.95,
    benchmark_sr: float = 0.0,
) -> float:
    """Minimum Track Record Length (months) per Bailey & Lopez de Prado (2014)."""
    z_alpha = stats.norm.ppf(confidence)
    sr_excess = sharpe_annual - benchmark_sr
    if sr_excess <= 0:
        return float("inf")

    sr_m = sharpe_annual / np.sqrt(12)
    sr0_m = benchmark_sr / np.sqrt(12)
    sr_excess_m = sr_m - sr0_m

    numer = (z_alpha ** 2) * (1 - skewness * sr_m + (excess_kurtosis / 4) * sr_m ** 2)
    denom = sr_excess_m ** 2
    mintrl = numer / denom if denom > 0 else float("inf")
    return mintrl

In [ ]:
mintrl = min_track_record_length(
    sharpe_annual=net_metrics["sharpe"],
    skewness=net_metrics["skewness"],
    excess_kurtosis=net_metrics["excess_kurtosis"],
    confidence=0.95,
    benchmark_sr=0.0,
)

print(f"Net Sharpe (annualized): {net_metrics['sharpe']:.4f}")
print(f"Skewness: {net_metrics['skewness']:.4f}, ExcessKurt: {net_metrics['excess_kurtosis']:.4f}")
print(f"MINTRL_95pct = {mintrl:.1f} months ({mintrl / 12:.1f} years)")
print(f"Observed track record: {net_metrics['n_months']} months")
observed_sufficient = net_metrics["n_months"] >= mintrl
print(f"Track record sufficient for 95% confidence: {'YES' if observed_sufficient else 'NO'}")

MinTRL is approximately 77.5 months at $M=1$ (no trial penalty), compared to our observed track record of 67 months. The track record falls short by about 10.5 months. Under the $M=10$ trial penalty applied in the lecture's Section 5, the MinTRL extends to 174 months (14.5 years) — the strategy cannot confirm its own Sharpe ratio at the trial count we actually used.

The excess kurtosis of 4.23 is the dominant driver. Under a Gaussian assumption (excess kurtosis = 0), the same Sharpe over the same period would need only about 10 months for confirmation. The heavy tails inflate MinTRL by roughly an order of magnitude. This is the single most important number in the report: it says "your data is too short and too fat-tailed to confirm what you think you found."

### Capital Allocation Discussion

The final section of the report is qualitative — the kind of language a quant researcher writes in the deployment memo.

In [ ]:
print("Strategy viability at scale: The base-regime net Sharpe of "
      f"{net_metrics['sharpe']:.2f} assumes a tiered spread model "
      "(5/15 bps for large/mid-small cap), but market impact grows with AUM "
      "via the sqrt-law: doubling capital raises impact costs by ~41%. "
      "At moderate AUM (< $200M), this strategy may remain viable; "
      "beyond $500M the impact drag likely erodes the edge entirely.")

In [ ]:
print("Position sizing discipline: Kelly criterion suggests fractional "
      "sizing: f* = SR / SR_max, where SR_max is approx SR + sigma_SR. "
      f"With net SR={net_metrics['sharpe']:.2f} and observed "
      f"MaxDD={net_metrics['max_dd']:.1%}, a half-Kelly allocation "
      "limits ruin probability while preserving compounding. "
      "Do NOT run at full Kelly on a strategy with PBO > 0.20.")

Position sizing connects the statistical evaluation to the capital allocation decision. The Kelly criterion provides the theoretically optimal bet size, but full Kelly assumes the Sharpe ratio is known with certainty -- an assumption we have already shown is unjustified given the MinTRL shortfall. Half-Kelly is the standard institutional compromise: it sacrifices roughly 25% of the growth rate but reduces drawdown risk substantially. The deployment decision below synthesizes all three layers into a single actionable recommendation.

In [ ]:
print("Deployment decision framework: The strategy clears a Sharpe > 0.5 "
      "threshold net of realistic costs, but the MinTRL "
      f"({mintrl:.0f} months at 95% confidence) compared to observed "
      f"track record ({net_metrics['n_months']} months) "
      "governs the deployment decision. "
      "A paper-trading period of 12 months before live allocation is recommended "
      "to confirm out-of-sample performance persistence and refine cost assumptions "
      "with actual fill data.")

---

## What You Built

The pipeline you just constructed — PurgedKFold, TransactionCostModel, and the three-layer evaluation report — is structurally identical to what a first-year quant researcher produces at an institutional fund. The specific implementation differs (production uses proprietary data, institutional TC models calibrated from NYSE TAQ fills, and multi-year track records with hundreds of model variants), but the logic is universal: purge the leakage, price the friction, evaluate the overfitting risk, and compute how long you need to wait before the evidence is convincing.

The results are sobering but honest. The GBM alpha model shows net Sharpe of 0.676 under base-regime costs — modestly above the typical institutional hurdle of 0.50. The DSR of 0.504 at $M=10$ is a marginal pass that flips to failure under slightly different cost assumptions. The PBO of 0.267 suggests the model ranking is stable but the absolute signal is statistically indistinguishable from zero at conventional confidence levels. The MinTRL of 77.5 months exceeds the observed 67-month track record by 10.5 months — the strategy cannot yet confirm its own Sharpe ratio.

The honest verdict: **wait**. Continue paper trading for approximately one more year, accumulate the additional months of out-of-sample evidence, and recalibrate the TC model with actual fill data before committing capital. This is not a failure — it is the expected outcome for a monthly cross-sectional alpha strategy with heavy-tailed returns on a survivorship-biased universe. The responsible backtest report is not designed to make the strategy look good; it is designed to make the truth clear.

The pipeline you built here handles the happy path — clean data, consistent model, well-behaved rebalancing. In production, the unhappy path is 80% of your week: corporate actions that break the price series, universe changes that scramble the cross-section, regime shifts that invert the signal, and execution failures that introduce tracking error. The tools from this week give you the evaluation framework; the tools from Week 14 (regime detection) and Week 6 (portfolio construction and risk) give you the machinery to handle the rest.

### Career Connection

The three-layer evaluation structure maps directly to institutional practice. At a multi-strategy hedge fund like Citadel or Millennium, a pod PM reviews strategy memos in exactly this format: Layer 1 is the performance summary that decides whether to keep reading; Layer 2 is the overfitting assessment that the risk team audits; Layer 3 is the deployment plan that determines capital allocation and monitoring. A quant researcher who can produce this report from scratch — not just run a pre-built tool but understand why each layer exists and what it catches — has demonstrated the core competency that separates "can fit a model" from "can evaluate a signal."

The `TransactionCostModel` you wrote is a simplified version of what Barra (now MSCI) sells as part of their risk and performance attribution suite, and what internal execution analytics teams build at firms like Two Sigma and D.E. Shaw. The `PurgedKFold` is the cross-validation primitive that underpins every backtest at shops that take Lopez de Prado's framework seriously. And the MinTRL computation is the number that determines whether a PM allocates $50M or $0 to your idea.

### Bridge to Next Week

Week 6 asks the next question: given a strategy that has survived rigorous evaluation, how should it be sized and combined with other strategies in a portfolio? The evaluation framework from this week flows directly into portfolio-level risk budgeting — because a strategy that passes the responsible backtest still needs to be sized correctly relative to its risk, its correlation with other positions, and the fund's overall risk appetite. You now know how to evaluate a single signal. Next week, you learn how to build a portfolio.